# Calculando Integral com o Método de Monte Carlo.

Dada a seguinte função Θ, podemos calcular uma integral aproximada usando o método de **Monte Carlo**.

$$ Θ = \int_0^1 g(u)du $$

Consideremos a variável aleatória $X$, onde sua distribuição segue a distribuição uniforme contínua:
 $X\sim Uniforme(0,1) $.

Por enquanto, vamos guardar na nossa mente a seguinte expressão: $$\int_0^1 g(u)f_X(u)du \tag{expressão 1}$$
A função de distribuição de probabilidade de uma distribuição uniforme é dada por

$$f_X(x) = \begin{cases} \dfrac{1}{b-a}, & a \le x \le b, \\0, & \text{caso contrário}. \end{cases}$$

Sabendo que $E_X{[g(x)]} = \int_{-∞}^{∞}g(x)f(x) dx$, notemos que **expressão 1** é equivalente a esperança de $g(u)$ dada a distribuição uniforme. Notemos também que $f_X(u) = 1$, por que $u \in [0, 1]$.

Assim, $$E_X{[g(u)]} = \int_0^1 g(u)*1\ du $$
 
Mas o que é a esperança de $g(X)$, se não a representação da média dos valores realizados de $g(X)$. Dessa forma:

$$\Theta =\int_0^1 g(u)\ du ≈ \frac{\sum_{i=1}^n{g(u_i)}}{n} $$

In [1]:
using Statistics

const lcg_step(multiplier, prior, increment, modulando) = begin
    (multiplier * prior + increment) % modulando
end

function generate_random!(random::Array{Float64}, n)
    @assert 0 < n ≤ length(random) "n can't be greater then the size of the point list."
    x1 = 274212
    for i ∈ 2:n
        x1 = lcg_step(x1, 305641, 30321, 11420107)
        random[i] = (x1 / 11420107)
    end
    random
end

function monte_carlo_integral(expr, steps)
    random_points = zeros(steps)
    generate_random!(random_points, steps)
    realizes = expr.(random_points)
    expect = Statistics.mean(realizes)
end

monte_carlo_integral (generic function with 1 method)

In [2]:
my_expr(x) = x+1
monte_carlo_integral(my_expr, 1000)

1.5017249930320267

No exemplo acima calculamos $\int_0^1x+1\  dx = 1.5$, no caso o valor para 20 steps foi $1.50172499$.

Mas e se quisermos calcular a integral para um intervalo diferente de $[0,1]$?

## Transformação de uma integral em $[a,b]$ para $[0,1]$.

Queremos encontrar a relação:
$$\int_a^bf(x)dx =\int_0^1g(u)du $$

Vamos começar achando uma transformação para $x$. $$t = \frac{x-a}{b-a} ⟹ x= a+t(b-a)$$

Assim, $ x=a ⟹ t = 0\ ,\  x=b ⟹ t = 1$

$$x = a + t(b-a)\\\frac{dx}{dt} = b-a ⟹ dx = (b-a)dt $$

Substituindo na equação original, **levando em consideração que nessa transformação, $a⟹0$ e $b⟹1$**.

$$\int_a^bf(x)dx ⟹ \int_0^1f(a+t(b-a)) (b-a)dt $$

---

Assim, voltando no exemplo anterior. E resolvendo de forma determínistica.


$$\int_{-5}^8x+1\ dx ⟹ \int_0^1 \big(-5+t(8-(-5))+1\big) (8-(-5))\ dt$$

$$\int_0^1 (13t-4)*13\ dt \implies \int_0^1 169t-52\ dt $$

$$ \bigg(\frac{169t²}{2}-52t\bigg)\Bigg|_0^1 = 32.5 $$

In [3]:
my_expr(t) = (13*t-4)*13
monte_carlo_integral(my_expr, 10000)

32.801922527319576

## Usando o Método de Monte Carlo para calcular Integrais Impróprias.

Não podemos usar a transformação anterior $t=\frac{t-a}{b-a}$, a transformação de intervalo $[a,b]⟶[0,1]$ não se traduz para $[0,∞) ⟶ [0, 1]$.

Vamos encontrar então outra transformação. Quero encontrar uma função $t=t(x)$ tal que $domínio(t) = [0,∞)$ e $imagem(t) = [0,1]$.

Dessa forma temos que encontrar um $t(x)$ que siga as seguintes propriedades.
$$x=0 ⟹ t(x) = 0\\ x⟶∞ ⟹ \lim_{x→∞}t(x) = 1$$

Dentre todas as funções $t(x)$ possíveis, será mais fácil encontramos aquelas da forma: $$t(x)=1-t'(x)$$

$$x=0 ⟹ t'(x) = 1 \\ T = \Big\{c^x, \frac{x}c\Big\} $$
$$ x⟶∞ ⟹ \lim_{x⟶∞}{t'(x)} = 0 \\ T = \Big\{c^{-x}, \frac{c}{x+1}\Big\} $$
Onde $T$ são os candidatos às expressões de $t'(x)$.

Com base nos candidatos podemos encontrar

$$ t'(x) = e^{-x}\ ou\ t'(x) = \frac{1}{x+1} $$

Logo, como função de transformação, podemos ter $$t(x) = 1 - e^{-x} $$

$$ e^{-x} = 1 - t ⟹ ln(e^{-x}) = ln(1-t) ⟹ x = -ln(1-t) $$
$$ \frac{dx}{dt} =  \frac{1}{1-t} $$

Assim, se quisermos calcular $ \int_0^∞f(x)\ dx $:
$$ \int_0^1 \frac{f(-ln(1-t))}{1-t} dt $$

### Exemplo, calculando de forma determinística.

$$ \int_0^∞e^{-x}dx ≡ \int_0^1 \frac{e^{-(-ln(1-t))}}{1-t} dt = 1 $$

In [7]:
my_expr(t) = ℯ^(log1p(-t))/(1-t)
monte_carlo_integral(my_expr, 32)

1.0